# Naive Bayes text classifier

In this exercise, you'll implement a Naive Bayes classifier for text from scratch.

In [1]:
import ipytest
from typing import List

ipytest.autoconfig()

### Training the model

  - Calculate $P(y)$ for each class label in the training data
  - Calculate $P(x_i|y)$ for each feature (term) for each class label in the training data using Laplace (add-one) smoothing
  
$$P(x_i|y)=\frac{c_{i,y} + 1}{c_i + m}$$

where 
  - $c_{i,y}$ is the number of times term $x_i$ appears in class $y$
  - $c_i$ is the total number of times term $x_i$ appears in the collection
  - $m$ is the number of classes


### Applying the model

Return the class $y \in Y$ that maximizes $P(y) \prod_{x_i} P(x_i|y)$.

Note that we need to consider $x_i$ at each *word position* in the document. Thus, we need to multiply with $P(x_i|y)$ as many times as $x_i$ appears in the document.
We can rewrite it as: $$P(y|x) \propto P(y) \prod_{i \in d} P(x_i|y)^{c_{i,d}}$$ where $c_{i,d}$ is the number of times term $i$ appears in document $d$.

Finally, we perform the computations in the log domain, that is, $$\log P(y) +  \sum_{i=1}^n (c_{i,d} \log P(x_i|y))$$

## 1) Probability estimations

The estimation of probabilities $P(x_i|y)$ and $P(y)$ are refactored to a separate class to make them testable.

In [2]:
class NBProbabilityEstimator:
    
    def get_prior_prob(self, y: int, training_labels: List[int]) -> float:
        """Computes the class prior probability, P(y).
        
        Args:
            y: Class ID.
            training_labels: Class labels in training data.
            
        Returns:
            The probability P(y).
        """
        return training_labels.count(y) / len(training_labels)
    
    def get_term_prob(self, count_inclass: int, count_total: int, num_unique_terms: int) -> float:
        """Computes the smoothed term probability for a given class, P(x_i|y).
        
        Args:
          count_inclass: Number of times the term appears in the given class.
          count_total: Number of times the term appears in the collection.
          num_unique_terms: Size of the vocabulary.
          
        Returns:
          The probability P(x_i|y).
        """
        return (count_inclass + 1) / (count_total + num_unique_terms)

### Tests

In [3]:
%%run_pytest[clean]

def test_prior_prob():
    nbpe = NBProbabilityEstimator()
    assert nbpe.get_prior_prob(1, [0, 1, 2, 3]) == 0.25
    assert nbpe.get_prior_prob(1, [1, 1, 2, 3]) == 0.5

def test_term_prob():
    nbpe = NBProbabilityEstimator()
    assert nbpe.get_term_prob(5, 20, 10) == 0.2
    assert nbpe.get_term_prob(74, 90, 10) == 0.75
    assert nbpe.get_term_prob(0, 6, 10) == 0.0625

..                                                                                 [100%]
2 passed in 0.01s


## 2) Naive Bayes classifier

Implement training and prediction for a Naive Bayes classifier.  We are operating with dense matrices for simplicity.

In [4]:
import numpy as np
import math

class NBClassifier:

    def __init__(self) -> None:
        self._nbprob = NBProbabilityEstimator()
        self._num_classes = 0
        self._prior_prob = None  # Holds P(y) values
        self._term_prob = None  # Holds P(x_i|y) values
        
    
    def fit(self, X_train: List[List[int]], y_train: List[int]) -> None:
        """Fits the model.
        
        Args:
          X_train: Document-term matrix for training data. 
              Rows correspond to documents and columns correspond to terms.
          y_train: Class labels corresponding to training documents.
        """        
        self._num_classes = len(np.unique(y_train))
        num_docs = len(X_train)
        num_terms = len(X_train[0])        
        self._term_prob = np.zeros((num_terms, self._num_classes))
        
        # Iterating through the vocabulary
        for i in range(num_terms):
            # Holds c_{i,j} values, i.e., the number of times term i appears with class j.
            class_count = [0] * self._num_classes
            for d in range(num_docs):
                class_count[y_train[d]] += X_train[d][i]
                        
            # Calculate P(x_i|y)
            total_count = sum(class_count)
            for j in range(self._num_classes):
                self._term_prob[i, j] = self._nbprob.get_term_prob(class_count[j], total_count, num_terms)
                
        # Pre-compute class prior probabilities
        self._prior_prob = []
        for y in range(self._num_classes):
            self._prior_prob.append(self._nbprob.get_prior_prob(y, y_train))

                
    def _predict_instance(self, x: List[int]) -> int:
        """Predict class for a single instance (document).
        
        Args:
          x: Document term vector.
          
        Returns:
          The predicted class label (0-indexed).
        """
        probs = []
        
        for y in range(self._num_classes):
            p = math.log(self._prior_prob[y])
            for i in range(len(x)):
                if x[i] > 0:
                    p += x[i] * math.log(self._term_prob[i][y])
            probs.append(p)
            
        # Get the class with the highest probability.
        return probs.index(max(probs))
        
    
    def predict(self, X_test: List[List[int]]) -> List[float]:
        """Make predictions for a set of documents.
        
        Args:
          X_test: Document-term matrix for test data.
          
        Returns:
          List with predictions.
        """
        return [self._predict_instance(x) for x in X_test]        

## 3) Testing on real data

We will be using a subset of the 20Newsgroups collection.

In [5]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "soc.religion.christian", 
    "talk.religion.misc",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware"
]

train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=123)
test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=123)

### Feature extraction

Get term frequencies using `CountVectorizer`. (We ignore terms that appear in less than 10 documents to speed up computation.)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=10)
X_train_counts = count_vect.fit_transform(train.data)
X_test_counts = count_vect.transform(test.data)

### Train and apply model

Note that we convert sparse matrices to dense ones. This is not efficient and should be avoided when working with large datasets. Nevertheless, this simplifies the implementation for this exercise.

In [7]:
nb = NBClassifier()
nb.fit(X_train_counts.toarray(), train.target.tolist())
predicted = nb.predict(X_test_counts.toarray())

### Evaluation

In [8]:
from sklearn import metrics

print(f"{metrics.accuracy_score(test.target, np.asarray(predicted)):.3f}")

0.595


**TODO** Once you completed the next exercise (E2-4), check back here to see if the performance you got with the implementation from scratch is comparable to that of sklearn. Most likely, you'll see quite a bit difference. Can you find out the reason for that? You can share the solution at the next class session (for a bonus point).

## Optional exercises

If you're done, try to implement it without making a conversion to dense matrices.

Also, do we really need to precompute and store all term probabilities?